In [1]:
# ## inputs
water_xyz = water = """3

H\t0.7493682\t0.0\t0.4424329
O\t0.0\t0.0\t-0.1653507
H\t-0.7493682\t0.0\t0.4424329     
"""
# geometry = water_xyz[3=]

methane = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

geometry = methane

In [2]:
from nbed import nbed
from pathlib import Path

geometry= str(Path('water.xyz').absolute())
n_active_atoms= 2
basis= 'STO-3G'
xc_functional= 'b3lyp'
output= 'openfermion'
projector= 'huzinaga'
localization= 'spade'
convergence= 1e-9
transform= 'jordan_wigner'


driver = nbed(
    geometry=geometry ,
    n_active_atoms=n_active_atoms,
    basis=basis,
    xc_functional=xc_functional,
    output=output,
    projector=projector,
    convergence=convergence,
    localization= localization,
    transform=transform,
    )


/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


AttributeError: 'SPADELocalizer' object has no attribute '_local_basis_transform'

In [ ]:
from nbed.ham_builder import HamiltonianBuilder

qham = HamiltonianBuilder(
    scf_method=driver.embedded_scf,
    constant_e_shift=driver.classical_energy,
    transform=transform,
).build(n_qubits=qubits)

converter = HamiltonianConverter(qham)
hamiltonians = getattr(converter, output.lower())

print_summary(hamiltonians, driver, transform, full_system=False)

In [ ]:
from nbed.embed import get_qubit_hamiltonian

qubit_H_mu = get_qubit_hamiltonian(exp_obj.molecular_ham_MU, 'jordan_wigner')
qubit_H_huz = get_qubit_hamiltonian(exp_obj.molecular_ham_HUZ, 'jordan_wigner')

In [ ]:
mu_classical_E =  exp_obj.classical_energy_MU
huz_classical_E =  exp_obj.classical_energy_HUZ

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [ ]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [ ]:
import numpy as np
mu_full_quantum = eig_vals_emb_mu[0] + mu_classical_E
huz_full_quantum = eig_vals_emb_huz[0] + huz_classical_E

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

In [ ]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state